In [1]:
import torch

torch.manual_seed(0)
torch.cuda.manual_seed(0)
print(torch.__version__)

1.9.0


# 2.2.1 创建tensor

创建一个5x3未初始化的Tensor

In [5]:
x=torch.empty(5,3)
print(x)

tensor([[2.8026e-45, 4.5640e-41, 2.8026e-45],
        [4.5639e-41, 1.4013e-45, 0.0000e+00],
        [3.9236e-44, 0.0000e+00, 3.5386e+00],
        [4.5640e-41, 1.4013e-45, 4.5640e-41],
        [1.4013e-45, 0.0000e+00, 1.4013e-45]])


创建一个5x3的随机初始化的Tensor:

In [6]:
x=torch.rand(5,3)
print(x)

tensor([[0.4963, 0.7682, 0.0885],
        [0.1320, 0.3074, 0.6341],
        [0.4901, 0.8964, 0.4556],
        [0.6323, 0.3489, 0.4017],
        [0.0223, 0.1689, 0.2939]])


创建一个5x3的long型全0的Tensor:

In [7]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


直接根据数据创建:

In [8]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


还可以通过现有的Tensor来创建，此方法会默认重用输入Tensor的一些属性，例如数据类型，除非自定义数据类型。

In [11]:
x=x.new_ones(5,3,dtype=torch.float64)# 返回的tensor默认具有相同的torch.dtype和torch.device
print(x)

x=torch.randn_like(x,dtype=torch.float) #指定新的数据类型
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.6035,  0.8110, -0.0451],
        [ 0.8797,  1.0482, -0.0445],
        [-0.7229,  2.8663, -0.5655],
        [ 0.1604, -0.0254,  1.0739],
        [ 2.2628, -0.9175, -0.2251]])


我们可以通过shape或者size()来获取Tensor的形状:

In [12]:
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


注意：返回的torch.Size其实就是一个tuple, 支持所有tuple的操作。

# 2.2.2 数据操作

### 算数操作
#### 加法形式一

In [15]:
y=torch.rand(5,3)
print(x+y)

tensor([[ 0.8074,  1.4620,  0.7294],
        [ 1.3166,  1.5673,  0.5714],
        [ 0.0873,  3.8464, -0.4508],
        [ 0.4772,  0.6712,  1.9882],
        [ 3.1979,  0.0236,  0.3744]])


#### 加法形式二

In [16]:
print(torch.add(x,y))

tensor([[ 0.8074,  1.4620,  0.7294],
        [ 1.3166,  1.5673,  0.5714],
        [ 0.0873,  3.8464, -0.4508],
        [ 0.4772,  0.6712,  1.9882],
        [ 3.1979,  0.0236,  0.3744]])


#### 加法形式三 

In [17]:
#add x to y
y.add_(x)
print(y)

tensor([[ 0.8074,  1.4620,  0.7294],
        [ 1.3166,  1.5673,  0.5714],
        [ 0.0873,  3.8464, -0.4508],
        [ 0.4772,  0.6712,  1.9882],
        [ 3.1979,  0.0236,  0.3744]])


注：PyTorch操作inplace版本都有后缀"", 例如`x.copy(y), x.t_()`

### 索引
我们还可以使用类似NumPy的索引操作来访问Tensor的一部分，需要注意的是：索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改。

In [21]:
y = x[0,:]
y+=1
print(y)
print(x[0,:])

tensor([3.6035, 3.8110, 2.9549])
tensor([3.6035, 3.8110, 2.9549])


### 改变形状
用view() 来改变tensor的形状

In [32]:
y=x.view(15)
z=x.view(-1,5)#-1所指的维度可以根据其他维度的值推出来  X.view(-1)中的-1本意是根据另外一个数来自动调整维度
print(x.size(),y.size(),z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


注意view()返回的新tensor与源tensor共享内存，也即更改其中的一个，另外一个也会跟着改变。

In [27]:
x+=1
print(x)
print(z)

tensor([[6.6035, 6.8110, 5.9549],
        [3.8797, 4.0482, 2.9555],
        [2.2771, 5.8663, 2.4345],
        [3.1604, 2.9746, 4.0739],
        [5.2628, 2.0825, 2.7749]])
tensor([[6.6035, 6.8110, 5.9549, 3.8797, 4.0482],
        [2.9555, 2.2771, 5.8663, 2.4345, 3.1604],
        [2.9746, 4.0739, 5.2628, 2.0825, 2.7749]])


如果不想共享内存，推荐先用clone创造一个副本然后再使用view。

In [38]:
x_cp = x.clone().view(15)
x -= 1
print(x)
print(x_cp)

tensor([[ 2.6035,  2.8110,  1.9549],
        [-0.1203,  0.0482, -1.0445],
        [-1.7229,  1.8663, -1.5655],
        [-0.8396, -1.0254,  0.0739],
        [ 1.2628, -1.9175, -1.2251]])
tensor([ 3.6035,  3.8110,  2.9549,  0.8797,  1.0482, -0.0445, -0.7229,  2.8663,
        -0.5655,  0.1604, -0.0254,  1.0739,  2.2628, -0.9175, -0.2251])


另外一个常用的函数就是item()，它可以将一个标量Tensor转换成一个Python number：

In [40]:
x=torch.randn(1)
print(x)
print(x.item())

tensor([-1.1624])
-1.162441372871399


# 2.2.3 广播机制

In [41]:
x=torch.arange(1,3).view(1,2)
print(x)
y=torch.arange(1,4).view(3,1)
print(y)
print(x+y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


# 2.2.4 运算的内存开销

In [42]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before =id(y)
y=y+x
print(id(y) == id_before)

False


In [43]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before =id(y)
y[:]=y+x
print(id(y) == id_before)

True


In [44]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
torch.add(x, y, out=y) # y += x, y.add_(x)
print(id(y) == id_before)

True


# 2.2.5 Tensor和NumPy相互转换
#### numpy()和from_numpy()这两个函数产生的Tensor和Numpy array()实际是使用相同的内存，改变其中一个时另一个也会改变
### Tensor转Numpy

In [47]:
a=torch.ones(5)
b=a.numpy()#tensor 转为 numpy
print(a,b)

a+=1
print(a,b)

b+=1
print(a,b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]
tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]


### NumPy数组转Tensor

In [50]:
import numpy as np
a=np.ones(5)
b=torch.from_numpy(a)#numpy 转换为tensor
print(a,b)

a+=1
print(a,b)
b+=1
print(a,b)

[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.] tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
[3. 3. 3. 3. 3.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


直接用torch.tensor()将NumPy数组转换为Tensor，该方法总是会进行数据拷贝，返回的Tensor和原来的数据不再共享内存

In [52]:
#用torch.tensor() 转换时不会共享内存

c=torch.tensor(a)
a+=1
print(a,c)

[4. 4. 4. 4. 4.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


# 2.2.6 Tensor on GPU

In [54]:
#以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuda")      #GPU
    y=torch.ones_like(x,device=device) #直接创建一个在gpu上的Tensor
    x=x.to(device)                     #等价与 .to("cuda")
    z=x+y
    print(z)
    print(z.to("cpu",torch.double))    #to()还可以同时更改数据类型

tensor([2, 3], device='cuda:0')
tensor([2., 3.], dtype=torch.float64)
